In this notebook I am going to try and use the [Formosanbank](https://ai4commsci.gitbook.io/formosanbank/the-bank-architecture/corpora) resources to make a basic autocorrect prediction system for Amis.
Even assuming the corpora have enough word distribution to be useful for a frequency-based prediction system, I already see the following problems:
* I don't know any Amis, and will have difficulty checking correctness
* No existing test sets
* Although I'm not sure how similar the morphology of Amis and Truku are (probably not very), I have no idea how an autocorrect would work for a fusional (?) language. If 'blah' is go and 'klah' is went and 'glah' is will go, should there be some bias toward forms of this stem if the input is similar enough in certain ways? At that point it starts to bleed into a stemmer/tagger.

**Note**: I have already used some of the validation/sterilization tools in FormosanBank. According to the FormosanBank README it should have been a few simple checks and changes like standardizing all punctuation to single-spaced equivalents, but I am unclear what exactly was changed.

In [8]:
corpora_dir = "../FormosanBank/Corpora"
find_lang = "ami" # Amis
find_glotto = "cent2104"
find_dialect = "Coastal"

First, let's make sure we're finding the expected files.

In [9]:
import os
all_xmls = []
for root, dirname, filenames in os.walk(corpora_dir):
    for f in filenames:
        if f.endswith("xml"):
            all_xmls.append(os.path.join(root,f))
print(len(all_xmls))
print(all_xmls[:2])

17108
['../FormosanBank/Corpora/Wikipedias/XML/Seediq/Hnigan.xml', '../FormosanBank/Corpora/Wikipedias/XML/Seediq/Jingay_siang.xml']


In [10]:
lang_xmls = []
import xml.etree.ElementTree as ET
for filepath in all_xmls:
    tree = ET.parse(filepath)
    root = tree.getroot()
    # taken from formosanbank validate_xml.py
    lang = root.get("{http://www.w3.org/XML/1998/namespace}lang")
    if lang:
        lang = lang.lower()
    else:
        # print(f"{filepath} doesn't appear to have a [lang] attrib: {root.attrib}")
        continue
    glottocode = root.get("glottocode")
    dialect = root.get("dialect")
    if lang.lower() == find_lang.lower():
        if not glottocode and not dialect: # they're both None
            print(f"glotto: {glottocode} | dialect: {dialect} | file: {' '.join(filepath.split('/')[-5:])}")
            # we assume the language is correct
            lang_xmls.append(filepath)
        else:
            if glottocode:
                if glottocode.lower() == find_glotto:
                    lang_xmls.append(filepath)
            if dialect:
                if glottocode.lower() == find_dialect:
                    lang_xmls.append(filepath)

glotto: None | dialect: None | file: Corpora Presidential_Apologies XML Amis Amis.xml
glotto: None | dialect: None | file: Corpora ILRDF_Dicts XML Amis Amis.xml
glotto: None | dialect: None | file: FormosanBank Corpora Virginia_Fey_Dictionary XML Amis.xml


In [11]:
print(len(lang_xmls))

19


Now it looks like we have the right files for the language and dialect we want. There is gray area here in terms of langauge and dialect definition, but this should be good enough.

In [12]:
def get_sent_list(root) -> list[str]:
    sents = root.findall(".//S")
    texts = []
    for s in sents:
        form_children = []
        for child in s:
            if child.tag == "FORM":
                form_children.append(child)
            # there is 'standard' and 'original' forms
            if len(form_children) == 1:
                texts.append(form_children[0].text)
            else:
                for child in form_children:
                    kind = child.get("kindOf")
                    if kind == "standard":
                        texts.append(child.text)
    return texts

Let's test our new method. 

In [13]:
testfile = lang_xmls[0]
print(testfile)
root = ET.parse(testfile).getroot()
ret = get_sent_list(root)
print(len(ret))
print(root.get("doesnotexist"))

../FormosanBank/Corpora/ePark/XML/ep3_文化篇/Amis/Coastal_Amis.xml
2128
None


Great, now let's see how it works on our collected files. Hopefully it's all formatted correctly!

In [14]:
all_sents = []
for file in lang_xmls:
    tree = ET.parse(file)
    root = tree.getroot()
    file_text_as_list = get_sent_list(root)
    print(f"{' '.join(file.split('/')[-3:])} length: {len(file_text_as_list)}")
    all_sents += file_text_as_list

ep3_文化篇 Amis Coastal_Amis.xml length: 2128
ep2_文化篇 Amis Coastal_Amis.xml length: 2128
ep2_閱讀書寫篇 Amis Coastal_Amis.xml length: 3375
ep3_族語短文 Amis Coastal_Amis.xml length: 492
ep3_繪本平台 Amis Coastal_Amis.xml length: 5028
ep2_族語短文 Amis Coastal_Amis.xml length: 814
ep2_生活會話篇 Amis Coastal_Amis.xml length: 3096
ep3_圖畫故事篇 Amis Coastal_Amis.xml length: 172
ep3_句型篇國中 Amis Coastal_Amis.xml length: 2404
ep3_句型篇高中 Amis Coastal_Amis.xml length: 3456
ep2_情境族語 Amis Coastal_Amis.xml length: 4578
ep3_閱讀書寫篇 Amis Coastal_Amis.xml length: 3465
ep3_情境族語 Amis Coastal_Amis.xml length: 3084
ep3_生活會話篇 Amis Coastal_Amis.xml length: 3186
ep2_學習詞表 Amis Coastal_Amis.xml length: 5444
ep1_九階教材 Amis Coastal_Amis.xml length: 4618
XML Amis Amis.xml length: 132
XML Amis Amis.xml length: 21928
Virginia_Fey_Dictionary XML Amis.xml length: 8883


Hmm, it's a bit odd that the first two match exactly in length, and even though they're episode 2/3, they are also the same topic. I bet they're the same file accidentally - let's use a `set` to make sure we're not getting biased counts.

In [15]:
print(len(all_sents))
print(len(set(all_sents)))

78411
14103


Whoa, that's a huge difference! There's a good chance that the Virginia Fey and others had single-word dictionaries and vocabularies that got duplicated. Let's check and see what's actually getting duplicated.

In [16]:
counts= {}
for e in all_sents:
    if e in counts:
        counts[e] = counts[e] + 1
    else:
        counts[e] = 1
i = sorted(counts.items(), key=lambda x: x[1], reverse=True)

In [17]:
print(i[:20])

[('Cima kiso?', 51), ('masadak', 41), ('Cima ko ngangan iso?', 36), ('kaka', 34), ("romi'ad", 34), ("mafana'", 34), ('Papina ko salikaka iso?', 34), ('romadiw', 33), ('katayni', 33), ("Nga'ay ho kiso?", 32), ('adada', 30), ('tatodong', 29), ('folad', 29), ('anini', 29), ('kolong', 29), ('niyam', 29), ("riko'", 29), ('pising', 29), ('Talacowa kiso?', 29), ('masakero', 28)]


As expected, most are single words, but there are some sentences in there. This is a bit odd, but makes sense when you consider that most of our corpus is from an online language teaching platform. More than likely, a sentence is repeated multiple times throughout a 'teaching unit', and potentially across multiple units as well.

Regardless, we now have a set of sentences - our corpus! - and can go ahead with normal Autocorrect stuff!

In [18]:
small_sentence_list= list(set(all_sents))

In [40]:
corpus = []
for s in small_sentence_list:
    corpus += [w.strip('/\\ .",?!;:[]()<>#$%^&*-') for w in s.lower().split(' ') if w.strip(' \'/\\ -.,"?!;:[]()<>#$%^&*') != '']
dictionary = set(corpus) # "Dictionary" of known words

I'm not so confident about putting everything to lowercase, but this should work. I'm unclear if I should be stripping the single quote `'` since some languages use it to demarcate a glottal stop.
Anyway, at this point we have our corpus, and we have our set of all words that we've seen. Next we need to make a count for each word in the corpus, which will be used to predict which 'autocorrected' word is most likely.

In [41]:
wcount = {}
for w in corpus:
    if w in wcount:
        wcount[w] +=1
    else:
        wcount[w] = 1
m = sum(wcount.values())
wprobs = {k:wcount[k]/m for k in wcount} # probability of each word according to our corpus

In [47]:
print(f"Corpus has {len(corpus)} words")
print(f"Sanity check for full corpus: {str(len(corpus) == m)}")
print(f"Corpus has {len(dictionary)} unique words")
print(f"Sanity check for unique words: {str(len(wcount) == len(wprobs) == len(dictionary))}")
temp_print_list = list(dictionary)[:5]
temp_print_list += ['a', 'o', 'no'] # known common words
print(f"Testing words: {temp_print_list}")
print("")
for t in temp_print_list:
    print(f"Count for {t}: {wcount[t]}")
    print(f"Prob  for {t}: {wprobs[t]}")
    print("")

Corpus has 108582 words
Sanity check for full corpus: True
Corpus has 13340 unique words
Sanity check for unique words: True
Testing words: ['dahecong', "ma'opiray", 'a', 'pitaw', "'alomaloman", 'a', 'o', 'no']

Count for dahecong: 1
Prob  for dahecong: 9.209629588697943e-06

Count for ma'opiray: 1
Prob  for ma'opiray: 9.209629588697943e-06

Count for a: 6336
Prob  for a: 0.05835221307399016

Count for pitaw: 11
Prob  for pitaw: 0.00010130592547567737

Count for 'alomaloman: 5
Prob  for 'alomaloman: 4.604814794348971e-05

Count for a: 6336
Prob  for a: 0.05835221307399016

Count for o: 3189
Prob  for o: 0.02936950875835774

Count for no: 5135
Prob  for no: 0.047291447937963936



The above looks good, although I am a little bit suspicious of how high the counts are for `a` `no` and `o`. I suppose with over 100k words in the corpus this isn't that high.
Now we will actually do autocorrect prediction. First, let's make a list of 'test' words - I'll make up a few to test based on real words from our dictionary. This would be a good thing to automate, where I find N words of various lengths and mangle them based on insertion, deletion, and swap, but for now I'll manually write the list.

In [48]:
# Key: input word to predict
# value: expected/hoped for output
words_to_test = {
    'midalof': 'misalof', # s->d
    'caykanac': 'caykanca', # missing -a
    'inra': 'nira', # swap n-i 
    'kici': 'kicic', # missing -c 
    'noo': 'no', # additional o 
    'asdfjkal': 'UNK', # keyboard mash 
    "'atongol": "'atongol", # same - make sure we include the original words if they exist
    "kasarramoramod": "kasaramoramod", # additional r
    "niyaro": "niyaro'", # missing '
    'x': 'a', # single-letter replacement; most likely should alwasy be 'a' or 'o'
    }


In [49]:
import nltk
from nltk.metrics.distance import edit_distance, jaccard_distance
from nltk.util import ngrams

In [50]:
# Takes in an input to correct, and a metric to calculate distance to word in our dictionary
# Returns a sorted dict of candidate words with the lowest distance
def suggest(istr, metric):
    candlist = {} # key: candidate word; value: distance
    for w in dictionary:
        # if w[0] == istr[0]:
            if w == istr:
                candlist[istr] = 0
            if metric == 'jaccard':
                # can't compute bigrams for single letter words
                if len(istr) == 1:
                    continue
                # 2-char bigrams of our strings
                i2grams = set(ngrams(istr,2))
                w2grams = set(ngrams(w,2))
                # calculate the distance between the two sets
                dist = jaccard_distance(i2grams, w2grams)
                # words are unique, distance may not be
            elif metric == 'edit': # metric == 'edit'
                dist = edit_distance(istr, w)
            # below doesn't work - division is too powerful for small probabilities
            elif metric == 'prob':
                dist = probs[w]/edit_distance(istr,w)
            else: # we didn't pass in a metric, oops
                raise ValueError("You forgot to pass in a metric!")
            candlist[w] = dist
    sorted_dict = {k:candlist[k] for k in sorted(candlist, key=candlist.get)}
    return sorted_dict

The above is a bit hacky, but right now I'm more interested in comparing the performance of the different metrics on our dataset. Ideally there would be a combination of multiple metrics, and statistical prediction of what the next word is likely to be based on word-level 2-grams (or n-grams).

In [51]:
for testword in words_to_test:
    expected = words_to_test[testword]
    # get the top X words according to our metric
    # having a threshold would be better, like n steps away according to edit distance etc.
    get_x_words = 10

    # get words suggested by jaccard metric
    jsug = list(suggest(testword, 'jaccard'))[:get_x_words] 
    jprobs = {w: wprobs[w] for w in jsug}
    # sort jaccard words by probability
    jprobs_sorted = {k: jprobs[k] for k in sorted(jprobs, key = jprobs.get, reverse=True)}
    
    # do the same for edit distance
    esug = list(suggest(testword, 'edit'))[:get_x_words]
    eprobs = {w: wprobs[w] for w in esug}
    eprobs_sorted = {k: eprobs[k] for k in sorted(eprobs, key = eprobs.get, reverse=True)}

    print(f"Testing {testword}, expected {expected}")
    print("\t" + f"Found: {str(expected in jsug)}" + "\n\t" + f"jaccard: {str(jsug)}")
    print("\t" + f"Found: {str(expected in jprobs_sorted)}" + "\n\t" + f"jprob: {str(jprobs_sorted)}")
    print("\t" + f"Found: {str(expected in esug)}" + "\n\t" + f"edit: {str(esug)}")
    print("\t" + f"Found: {str(expected in eprobs_sorted)}" + "\n\t" + f"eprob: {str(eprobs_sorted)}")
    print("\n\n\n")


Testing midalof, expected misalof
	Found: True
	jaccard: ['misalof', 'cidal', 'alofo', 'alomi', 'falofalo', 'lalosidan', 'midadiwal', 'tadalosid', 'midawa', 'milalo']
	Found: True
	jprob: {'cidal': 0.0008196570333941169, 'lalosidan': 0.000322337035604428, 'misalof': 5.525777753218766e-05, 'tadalosid': 2.762888876609383e-05, 'midawa': 1.8419259177395885e-05, 'alofo': 9.209629588697943e-06, 'alomi': 9.209629588697943e-06, 'falofalo': 9.209629588697943e-06, 'midadiwal': 9.209629588697943e-06, 'milalo': 9.209629588697943e-06}
	Found: True
	edit: ['misalof', 'masalof', 'milalo', 'midadoy', 'mipalo', 'mifalod', "midafo'", 'mitalod', 'midaho', 'mifalah']
	Found: True
	eprob: {'misalof': 5.525777753218766e-05, 'mipalo': 3.683851835479177e-05, 'midaho': 3.683851835479177e-05, 'mifalah': 3.683851835479177e-05, 'masalof': 2.762888876609383e-05, 'milalo': 9.209629588697943e-06, 'midadoy': 9.209629588697943e-06, 'mifalod': 9.209629588697943e-06, "midafo'": 9.209629588697943e-06, 'mitalod': 9.209629

# Conclusion
From the above, it appears that it works okay for now, although there are a few issues. For now these are my thoughts.
1. Jaccard fails for single-letter words. (It's fixed now, but I had to go back and add a check to skip Jaccard for single letters.) This is expected, but good to know.
2. Shorter words have difficulty when the error is a swap. This is because for jaccard distance there are too few bigrams to do meaningful comparison when the word is short; for edit distance the swap is counted as 2 moves (I think), and short words will be equidistant from many other words by virtue of being a potential substring.
3. It is useful to sort the candidate words based on probability. Ideally there should be some sort of computation based on distance, probability, and even word prediction.
4. I missed some full-width punctuation in my string sterilization, as well as a few symbols (I've fixed some now).